<a href="https://colab.research.google.com/github/CUAI-Deeplearning/Pytorch-NLP-Study/blob/stats/%EA%B0%95%ED%98%B8%ED%98%84/Chapter13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 13. 듀얼리티의 활용
## 13.1 들어가며

* 듀얼리티(duality)
  * 머신러닝을 통해 특정 도메인의 데이터 X를 받아 다른 도메인의 데이터 Y로 맵핑하는 함수를 근사
  * 대부분의 머신러닝 데이터셋은 두 도메인 사이의 데이터로 구성된다.(오디오, 텍스트)

  <img src="https://github.com/CUAI-Deeplearning/Pytorch-NLP-Study/blob/stats/%EA%B0%95%ED%98%B8%ED%98%84/asset/chap13/image1.png?raw=1" width="50%" height="50%">


* 듀얼리티 = 두 도메인 사이의 관계(음성 인식, 음성 합성)
  * 대부분 머신러닝 문제가 가지는 듀얼리티 속성
  * 기계번역 : 각 도메인 데이터 사이에 정보량 차이가 거의 없다(장점)
  * 듀얼리티 적극적으로 활용하면 기계번역 고도화 가능하다.

* 데이터에 따른 듀얼리티 사례

|$D_1$ 도메인 | $D_1 \rightarrow D_2$ | $D_1 \leftarrow D_2$ | $D_2$ 도메인 |
| :-------- | :---------------------| :-------------------- |:---------- |
| 문장 | 기계번역 | 기계번역 | 문장 |
| 음성 신호 | 음성 인식 | 음성 합성 | 문장 |
| 이미지 | 이미지 분류 | 이미지 합성 | 클래스|
| 문서 | 요약 | 본문 생성 | 문장 |
| 문장 | 텍스트 분류 | 문장 생성 | 클래스 |

### 13.1.2 CycleGAN

* 번역 이외 듀얼리티, 영상 처리에 관한 논문 CycleGAN
* cycleGAN
  * 짝지어지지 않은 두 도메인의 이미지가 있을 때, X 도메인의 이미지를 Y 도메인의 이미지로 변환 방법
  * 사진의 전체 구조 유지하되 유명한 화가 모네 풍으로 바꾸거나 말을 얼룩말 / 민무늬 변경
  * 겨울 풍경을 여름 풍경으로 변화 가능
  * 예제  
    <img src="https://github.com/CUAI-Deeplearning/Pytorch-NLP-Study/blob/stats/%EA%B0%95%ED%98%B8%ED%98%84/asset/chap13/image2.jpg?raw=1" width="100%" height="50%">

* $X\rightarrow{Y}$ 와 $Y\rightarrow{X}$ : 모두 각각 생성자 $G, F$ 와 판별자 $D_x, D_y$ 를 가지고 있어서 최소/최대화 게임 수행
* $G$ : $x$ 를 입력으로 받아 $\hat{y}$ 으로 변환
* $D_Y$ : $\hat{y}$ 또는 $y$ 를 입력으로 받아 합성한 이미지인지 원래 $Y$ 도메인의 이미지인지 예측
* $F$ : $y$ 를 입력으로 받아 $\hat{x}$ 으로 변환
* $D_x$ : $\hat{x}$ 또는 $x$ 를 입력으로 받아 합성 유부를 판단

<img src="https://github.com/CUAI-Deeplearning/Pytorch-NLP-Study/blob/stats/%EA%B0%95%ED%98%B8%ED%98%84/asset/chap13/image3.png?raw=1" width="100%" height="50%">

* $\hat{X}, \hat{Y}$를 합성할 때 기존의 도메인 X, Y에 실제로 속하는 이미지처럼 만들어야 한다.
* $\hat{X}, \hat{Y}$에서 원래 데이터로 돌아올 수 있어야 한다.
$\rightarrow$ 기계번역에 적용

## 13.2 듀얼리티를 활용한 지도학습
* 듀얼 지도학습(DSL, Dual Supervised Learning)
  * 기존의 teacher-forcing 문제로 생기는 어려움을 해결할 때 강화학습 대신 듀얼리티에서 regularization term 이끌어내 해결한다.
  * 베이즈 정리에 따른 수식(항상 성립)
  $$P(y|x) = {P(x|y)P(y)} \over {P(x)}$$
  $$P(y|x)P(x) = P(x|y)P(y)$$

* 이 수식대로 훈련한 모델은 다음 수식을 만족한다.
$$P(x)P(y|x;\theta_{y\rightarrow x}) = P(y)P(x|y;\theta_{y\rightarrow x})
$$
$$ \text{where } (x,y) \sim \mathcal{B} = \{x_i,y_i\}_{i=1}^N$$

* 이 전제를 번역 훈련을 위한 목표에 적용한다.

$$objective1: \text{min}_{\theta_{x\rightarrow y}} {1 \over n} \sum_{i=1}^n \ell_1(f(x_i;\theta{x\rightarrow y}),y_i), $$

$$objective2: \text{min}_{\theta_{y\rightarrow x}} {1 \over n} \sum_{i=1}^n \ell_1(g(x_i;\theta{y\rightarrow x}),x_i), $$

$$s.t. P(x) P(y|x;\theta_{x\rightarrow y}) = P(y) P(x|y; \theta_{y\rightarrow x}), \forall x,y $$

* 해석
  * 목표 1 : 베이즈 정리에 따른 제약 조건 만족함과 동시에 $\ell_1$을 최소화
    * $\ell_1$ : 번역함수 f에 입력 $x_i$를 넣어 나온 반환값과 $y_i$ 사이 손실
  * 목표 2 : $\ell_2$를 번역함수 $g$에 대해 같은 작업 수행하고 최소화하여 만족시킨다.
    * $\ell_2$ : 번역함수 g에 입력 $x_i$를 넣어 나온 반환값과 $y_i$ 사이 손실


$\mathcal{L}_{duality} = |\left(\log \hat{P}(x) + \log P(y|x;\theta_{x\rightarrow y})\right) - \left(\log \hat{P}(y) + \log P(x|y;\theta_{y\rightarrow x})\right) |^2$

* 베이즈 정리에 따른 제약 조건의 양 변의 값의 차이를 최소화하는 MSE 손실 함수
* 동시에 훈련시키는 신경망 가중치 파라미터 $\rightarrow$ 각각 $\log P(y|x;\theta_{x\rightarrow y}),\log P(x|y;\theta_{y\rightarrow x}) $ 구한다.
* 단일 언어 코퍼스로 미리 훈련시킨 언어 모델 $\rightarrow \log \hat{P}(x), \log \hat{P}(y)$ 근사 가능하다.

* 기존의 손실 함수에 추가하여 동시에 최소화

$$ \theta_{x \rightarrow y} \leftarrow \theta_{x \rightarrow y} - \gamma \triangledown_{\theta_{x \rightarrow y}} {1 \over n} \sum_{i=1}^n [\ell_1(f(x_i;\theta_{x\rightarrow y}),y_i) + \lambda_{x \rightarrow y} \mathcal{L}_{duality}]
$$


$$ \theta_{y \rightarrow x} \leftarrow \theta_{y \rightarrow x} - \gamma \triangledown_{\theta_{y \rightarrow x}} {1 \over n} \sum_{i=1}^n [\ell_2(g(y_i;\theta_{y\rightarrow x}),x_i) + \lambda_{y \rightarrow x} \mathcal{L}_{duality}]$$


* $\lambda$를 통해 손실함수 내 비율 조절 가능
  * $\lambda$ 는 하이퍼 파라미터, 0.01일 때 가장 좋은 성능을 보인다.
  * DSL 방식이 기존의 teacher-forcing, MLE / MRT 보다 더 높은 성능을 보인다.
  * Regularization term을 추가하여 훈련 까다롭고 비효율적인 강화학습보다 나은 성능을 보인다.

### 13.2.1 파이토치 예제 코드

In [0]:
# simple_nmt와 dual_trainer.py
X2Y, Y2X = 0, 1

In [0]:
def _reordering(self, x, y, l):
  indice = l.topk(l.size(0))[1]
  x_ = x.index_select(dim=0, index=indice).contiguous()
  y_ = y.index_select(dim=0, index=indice).contiguous()
  l_ = l.index_select(dim=0, index=indice).contiguous()

  restore_indice = (-indice).topk(l.size(0))[1]

  return x_,y_,l_, restore_indice

In [0]:
def _get_loss(self, x, y, x_hat, y_hat, x_lm=None, y_lm=None, lagrange=1e-3):

  losses = []
  losses += [self.crits[X2Y](y_hat.contiguous().view(-1, y_hat.size(-1)),
                             y.contiguous().view(-1)
                             )]
  losses += [self.crits[Y2X](x_hat.contiguous().view(-1, x_hat.size(-1)),
                             x.contiguous().view(-1)
                             )]

  losses[X2Y] = losses[X2Y].view(y.size(0), -1).sum(dim=-1)
  losses[Y2X] = losses[Y2X].view(x.size(0), -1).sum(dim=-1)

  if x_lm is not None and y_lm is not None:
    lm_losses = []
    lm_losses += [self.crits[X2Y](y_lm.contiguous().view(-1, y_lm.size(-1)),
                                  y.contiguous().view(-1)
                                  )]
    lm_losses += [self.crits[Y2X](x_lm.contiguous().view(-1, x_lm.size(-1)),
                                  x.contiguous().view(-1)
                                  )]
    
    lm_losses[X2Y] = lm_losses[X2Y].view(y.size(0), -1).sum(dim=-1)
    lm_losses[Y2X] = lm_losses[Y2X].view(x.size(0), -1).sum(dim=-1)

    dual_loss = lagrange * ((-lm_losses[Y2X] + -losses[X2Y].detach()) - \
                            (-lm_losses[X2Y] + -losses[Y2X].detach()))**2

    dual_loss_x2y = lagrange * ((-lm_losses[Y2X] + -losses[X2Y]) - \
                                (-lm_losses[X2Y] + -losses[Y2X].detach()))**2
    dual_loss_y2x = lagrange * ((-lm_losses[Y2X] + -losses[X2Y].detach()) - \
                                (-lm_losses[X2Y] + -losses[Y2X]))**2
    
    losses[X2Y] += dual_loss_x2y
    losses[Y2X] += dual_loss_y2x


  if x_lm is not None and y_lm is not None:
    return losses[X2Y].sum(), losses[Y2X].sum(), dual_loss.sum()
  else:
    return losses[X2Y].sum(), losses[Y2X].sum(), None

In [0]:
import numpy as np
import torch

from torch import optim
import torch.nn.utils as torch_utils
#from ignite.engine import Engine, Events

VERBOSE_SILENT = 0
VERBOSE_EPOCH_WISE = 1
VERBOSE_BATCH_WISE = 2
# 앞의 선언된 함수로 1 epoch 훈련

def train_epoch(self,
                train,
                optimizers,
                no_regularization=True,
                verbose=VERBOSE_BATCH_WISE
                ):
  
  total_loss, total_word_count = 0,0
  total_grad_norm = 0
  avg_loss, avg_grad_norm = 0,0
  sample_cnt = 0

  progress_bar = tqdm(train,
                      desc = 'Training: ',
                      unit = 'batch'
                      ) if verbose is VERBOSE_BATCH_WISE else train

  for idx, mini_batch in enumerate(progress_bar):
    optimizers[X2Y].zero_grad()
    optimizers[Y2X].zero_grad()
    
    x_0, y_0 = (mini_batch.src[0][:, 1:-1], # BOS, EOS 제거
                mini_batch.src[1] - 2
                ), mini_batch.tgt[0][:, :-1]

    y_hat = self.models[X2Y](x_0, y_0)

    with torch.no_grad():
      y_lm = self.language_models[X2Y](y_0)

    x_0, y_0_0, y_0_1, restore_indice = self._reordering(mini_batch.src[0][:, :-1],
                                                         mini_batch.tgt[0][:, 1:-1],
                                                         mini_batch.tgt[1] - 2
                                                         )
    y_0 = (y_0_0, y_0_1)

    x_hat = self.models[Y2X](y_0, x_0).index_select(dim=0, index=restore_indice)

    with torch.no_grad():
      x_lm = self.language_models[Y2X](x_0)

    x, y = mini_batch.src[0][:, 1:], mini_batch.tgt[0][:, 1:]
    losses = self._get_loss(x,
                            y,
                            x_hat,
                            y_hat,
                            x_lm,
                            y_lm,
                            lagrange=self.config.dsl_lambda if not no_regularization else .0
                            )
    losses[X2Y].div(y.size(0)).backward()
    losses[Y2X].div(x.size(0)).backward()
    
    word_count = int((mini_batch.src[1].detach().sum()) +
                     (mini_batch.tgt[1].detach().sum())
                     )
    loss = float(losses[X2Y].detach() + losses[Y2X].detach()) - float(losses[-1].detach() * 2)
    param_norm = float(utils.get_parameter_norm(self.models[X2Y].parameters()).detach() +
                       utils.get_parameter_norm(self.models[Y2X].parameters()).detach()
                       )
    grad_norm = float(utils.get_grad_norm(self.models[X2Y].parameters()).detach() +
                      utils.get_grad_norm(self.models[Y2X].parameters()).detach()
                      )
    total_loss += loss
    total_word_count += total_word_count
    total_grad_norm += total_grad_norm

    avg_loss = total_loss / total_word_count
    avg_grad_norm = total_grad_norm / (idx + 1)

    if verbose is VERBOSE_BATCH_WISE:
      progress_bar.set_postfix_str('|param|=%.2f |g_param|=%.2f loss=%.4e PPL= %.2f' % (param_norm,
                                                                                        grad_norm,
                                                                                        loss / word_count,
                                                                                        exp(avg_loss)
                                                                                        ))
      
    torch_utils.clip_grad_norm_(self.models[X2Y].parameters(),
                                self.config.max_grad_norm
                                )
    torch_utils.clip_grad_norm_(self.models[Y2X].parameters(),
                                self.config.max_grad_norm
                                )
    

    optimizers[X2Y].step()
    optimizers[Y2X].step()

    sample_cnt += mini_batch.tgt[0].size(0)

    if idx >= len(progress_bar) * self.config.train_ratio_per_epoch:
      break

    if verbose is VERBOSE_BATCH_WISE:
      progress_bar.close()

    return avg_loss, param_norm, avg_grad_norm

### 13.2.2 실험 결과

* BLEU 수치상 DSL이 가장 높은 성능을 보인다.(MLE < MRT < DSL)

* teacher-forcing 사용하면서 샘플링 기반 학습효율 저하 없이 간단한 regularization term을 손실함수에 더하여 더 나은 성능의 기계번역 모델을 학습할 수 있다.

## 13.3 듀얼리티를 활용한 비지도학습

### 13.3.1 듀얼 러닝 기계번역
* 직접 미분 값을 전달할 수 없으나 CycleGAN과 비슷한 개념이다.
* 병렬 코퍼스로 훈련된 기본 성능을 단일 코퍼스로 그 성능을 극대화하고자 한다.
* 그래디언트를 전달할 수 없으므로, 강화학습을 활용해 판별자 값을 전달해야 한다.

* 단일 언어 코퍼스에서 받은 문장 s 번역하고, 번역된 문장 $s_{mid}$를 사용해 반대 방향 번역을 통해 복원
* 복원된 문장 $\hat{s}$이 원래 처음 문장과의 차이 $\Delta(\hat{s},s)$가 최소화되도록 훈련
  * 번역된 문장 $s_{mid}$가 얼마나 자연스럽게 해당 언어의 문장이 되었는지 여부가 중요한 지표가 된다.

* 알고리즘
  * Language A, Language B 두 개 도메인 문장들이 주어진다.
  * 생성자 $G_{A \rightarrow B}$의 파라미터 $\theta_{AB}$, 반대 방향 생성자 $F_{B \rightarrow A}$의 파라미터 $\theta_{BA}$ : 병렬 코퍼스로 사전훈련된 상태
  * 폴리시 그래디언트를 통한 파라미터 업데이트

  $$\theta_{AB} \leftarrow \theta_{AB} + \gamma\triangledown_{\theta_{AB}} \hat{\mathbb{E}}[r]$$
  $$\theta_{BA} \leftarrow \theta_{BA} + \gamma\triangledown_{\theta_{BA}} \hat{\mathbb{E}}[r]$$

    * $\hat{\mathbb{E}}[r]$ 각각의 파라미터에 대해 미분한 값을 더해주는 것을 확인한다.
  * 보상의 기댓값
  $$r = \lambda \cdot r_{AB} + (1-\lambda) \cdot r_{BA},$$
  $$\text{ where } r_{AB} = LM_B(s_{mid})$$
  $$\text{and }r_{AB} = \log P(s|s_{mid};\theta_{BA})$$

    * k개의 샘플링 문장에 대해 각 방향에 대한 보상 각각 구하고 이를 선형결합한다.
    * $s_{mid}$ : 샘플링한 문장, $LM_{B}$로 해당 문장이 Language B의 집합에 어울리는지 보상값 계산
    * $LM_B$ : 언어 모델로 기존의 Language B의 단일 언어 코퍼스로 사전 훈련되어 있다.
      * 자연스러운 문장이 생성될수록 해당 언어 모델에서 높은 확률을 가질 것이다.
      * 다수의 단일 언어 코퍼스를 가지므로 언어 모델 만드는 것은 어렵지 않다.?

  $$\triangledown_{\theta_{AB}} \hat{\mathbb{E}} = {1 \over K} \sum_{k=1}^K [r_k \triangledown_{\theta_{AB}} \log P(s_{mid, k} | s; \theta_{AB})]
  $$

  $$\triangledown_{\theta_{AB}} \hat{\mathbb{E}} = {1 \over K} \sum_{k=1}^K [(1-\lambda) \triangledown_{\theta_{BA}} \log P(s | s_{mid, k} ; \theta_{BA})]
  $$


* 듀얼 러닝이 가장 뛰어난 성능 개선을 보인다.

* 문장의 길이와 상관없이 모든 구간에서 베이스라인 NMT를 압도한다.
  * 병렬 코퍼스의 양이 커질수록 단일 언어 코퍼스를 활용한 듀얼 러닝에 의한 성능 향상의 폭은 감소한다.
  * 강화학습과 듀얼리티 접목 $\rightarrow$ 적은 양의 병렬 코퍼스와 다수의 단일 언어 코퍼스 활용해 번역기 성능을 효과적으로 개선하였다.

  

### 13.3.2 듀얼 비지도학습
* 듀얼 지도학습(DSL) : 베이즈 정리에 따른 수식으로 제약 조건
* DUL : 주변 분포의 성질 이용해서 제약 조건 만든다.
  * 주변 분포
  $$ P(y) = \sum_{x \in \mathcal{X}} P(x, y) = \sum_{x \in \mathcal{X}} P(y|x) P(x)  $$

  * 위의 수식이 항상 참이다. 조건부 확률로 나타낼 수 있고, 기댓값 표현으로 바꿀 수 있다.
  * K번 샘플링해 몬테카를로 샘플링으로 근사하여 표현 가능.

$$ P(y) = \sum_{x \in \mathcal{X}} P(y|x;\theta) P(x) = \mathbb{E}_{X \sim P(X)}[P(y|x;\theta)]
$$
$$ \approx {1 \over K} \sum_{i=1}^K P(y|x^i ; \theta), \text{ where }x^i \sim P(x) $$


* 기계번역에 적용
  * N개 소스 문장x, 타깃 문장y로 구성된 양방향 병렬 코퍼스 $\mathcal{B}$
  * S개 타깃 문장y로만 구성된 단일 언어 코퍼스 $\mathcal{M}$ 가정

$$ \mathcal{B} = \{(x^n, y^n\}_{n=1}^N
$$
$$\mathcal{M} = \{y^s\}_{s=1}^S$$

* 조건 : 목적 함수를 최대화 + 주변 분포 제약 조건 만족

$$Objective : \sum_{i=1}^N \log P(y^n | x^n;\theta),$$
$$\text{s.t. }P(y) = \mathbb{E}_{x \sim P(x)} [P(y|x;\theta)], \forall y \in \mathcal{M}$$



* $\lambda$와 함께 $S(\theta)$로 표현하여 기존의 손실함수에 추가

$$ S(\theta) = [\log \hat{P}(y) - \log \mathbb{E}_{x \sim \hat{P}(x)} [P(y|x;\theta)]]^2
$$
$$\mathcal{L}(\theta) = - \sum_{n=1}^N \log P(y^n | x^n;\theta) + \lambda \mathcal{S}(\theta)$$
$$= - \sum_{n=1}^N \log P(y^n | x^n;\theta) + \lambda \sum_{s=1}^S[\log \hat{P}(y^s) - \log \mathbb{E}_{x \sim \hat{P}(x)}[P(y^s|x;\theta)]]^2
$$

* $\hat{P}(y)$ : 단일 언어 코퍼스로 만든 언어 모델을 통해 계산한 각 문장들의 확률값
  * 이를 통해 소스 문장 x를 샘플링하여 네트워크 $\theta$ 통과시켜 $P(y|x;\theta)$ 구해야하나 다른 접근 사용

* 중요도 표집을 통해 타깃 언어 문장 y를 반대 방향 번역기($y \rightarrow x$)에 넣어 K개의 소스 언어 문장 x를 샘플링하여 P(y) 구한다.

$$P(y) = \mathbb{E}_{x \sim \hat{P}(x)}[P(y|x;\theta)] = \sum_{x \in \mathcal{X}} P(y|x;\theta)\hat{P}(x)$$
$$= \sum_{x \in \mathcal{X}} {P(y|x;\theta)\hat{P}(x) \over P(x|y)} P(x|y)$$
$$= \mathbb{E}_{x \sim P(x|y)} \left[{P(y|x;\theta)\hat{P}(x)\over P(x|y)}\right]$$
$$\approx {1 \over K} \sum_{i=1}^K {{P(y|x_i;\theta)\hat{P}(x_i)} \over {P(x_i | y)}}, x_i \sim P(x|y)$$

* 위 과정을 하나의 손실 함수로 표현

$$\mathcal{L}(\theta) \approx -\sum_{n=1}^N \log P(y^n|x^n;\theta) + \lambda \sum_{s=1}^S [\log \hat{P}(y^s) - \log {1 \over K} \sum_{i=1}^K {\hat{P}(x_i) P(y^s|x_i \theta) \over P(x_i|y^s)}]^2$$

  * $\sum_{n=1}^N \log P(y^n|x^n;\theta)$ : 문장 $x^n$이 주어질 때 $y^n$의 확률을 최대로 하는 $\theta$ 찾는다.
  * 두 번째 항 : 단일 언어 코퍼스에서 주어진 문장 $y^s$ 언어 모델에서의 확률값 $\log \hat{P}(y^s)$와의 차이를 줄인다.
  * 반대 방향 번역기를 통해 K번 샘플링한 문장 $x_i$의 언어 모델 확률값 $\hat{P}(x_i), x_i$ 주어질 때, $y^s$ 확률값 곱하고, 문장 $y^s$가 주어질 때 샘플링한 문장 $x_i$의 확률값으로 나누어준 값이 된다.

* 듀얼 비지도학습(DUL)은 기존 단일 언어 코퍼스 활용하는 알고리즘 방법들보다 최종적으로 가장 뛰어남을 알 수 있다.
  * 듀얼러닝(DSL)보다 뛰어나고 불안정하고 비효율적인 강화학습 안쓰고도 더 나은 성능을 보인다.

## 13.4 쉬어가기: back-translation 재해석하기

* back-translation을 듀얼리티 관점에서 수식으로 해석해서 잘 동작하는지 설명
* N개의 소스 문장x, 타깃 문장y로 구성된 양방향 병렬 코퍼스 B
* S개의 타깃 문장 y로만 이루어진 단일 언어 코퍼스 M 가정
$$B = \{(x^n, y^n)\}_{n=1}^N$$
$$M = \{y^s\}_{s=1}^S$$

* 최소화하려는 손실 함수
$$\mathcal{L}(\theta) = -\sum_{n=1}^N \log P(y^n | x^n;\theta) - \sum_{s=1}^S \log P(y^s)$$

* P(y)를 주변 분포의 속성으로 표현  
$\log P(y) = \log \sum_{x \in \mathcal{X}} P(y|x) P(x)$  
$\phantom{\log P(y)} = \log \sum_{x \in \mathcal{X}} P(x|y) {P(y|x) P(x)\over P(x|y)}$  
$\phantom{\log P(y)} \ge \sum_{x \in \mathcal{X}} P(x|y) \log {P(y|x) P(x) \over P(x|y)}$  
$\phantom{\log P(y)} = \mathbb{E}_{x \sim P(x|y)} [\log P(y|x) + \log {P(x) \over P(x|y)}]$  
$\phantom{\log P(y)} = \mathbb{E}_{x \sim P(x|y)} [\log P(y|x)] + \mathbb{E}_{x \sim P(x|y)} \left[\log {P(x) \over P(x|y)}\right]$  
$\phantom{\log P(y)} = \mathbb{E}_{x \sim P(x|y)} [\log P(y|x)] - \text{KL} \left(P(x|y)\parallel P(x)\right)$  

* 젠센 부등식 정리로 항상 $ \log P(y)$보다 작거나 같은 수식으로 정리 가능
* 젠센 부등식(Jensen's inequality)
  * 로그의 함수 곡선 이용

* 이 성질로 증명이 가능하며 양변에 음의 부호를 취하면 다음과 같다.
$$-\log P(y) \le -\mathbb{E}_{x \sim P(x|y)} [\log P(y|x)] + \text{KL}(P(x|y) \parallel P(x))$$

* 위 부등호가 항상 성립하므로, $-\log P(y)$ 보다 항상 큰 수식을 최소화 한다.  
$\Leftrightarrow -\log P(y)$을 최소화한다.
* $\mathcal{L}(\theta)$ 에 이 수식을 대입한다.  
$\mathcal{L}(\theta) \le -\sum_{n=1}^N \log P(y^n|x^n;\theta)  -\sum_{s=1}^S(\mathbb{E}_{x \sim P(x|y^s)}[\log P(y^s | x;\theta)] - \text{KL}(P(x|y^s) \parallel P(x))) $
$\phantom{\mathcal{L}(\theta)} \approx -\sum_{n=1}^N \log P(y^n|x^n;\theta) - {1 \over K} \sum_{s=1}^S \sum_{i=1}^K \log P(y^s|x_i;\theta)+\sum_{s=1}^S \text{KL} (P(x|y^s) \parallel P(x))$  
$\phantom{\mathcal{L}(\theta)} = \tilde{\mathcal{L}}(\theta)$

* 결과적으로 $\tilde{\mathcal{L}}(\theta)$ 최소하하고자 경사하강법 수행하여 최적화
* 새로운 손실 함수를 $\theta$에 대해 미분하고, 쿨백-라이블러 발산(KLD)는 $\theta$에 대해 상수이므로 생략된다.  


$$\triangledown_{\theta} \tilde{\mathcal{L}}(\theta) = - \sum_{n=1}^N \triangledown_{\theta} \log P(y^n|x^n;\theta)-{1\over K} \sum_{s=1}^S \sum_{i=1}^K \triangledown_\theta \log P(y^s|x_i;\theta)$$

* 손실 함수 각 부분의 의미
  * 첫 번째 항($\sum_{n=1}^N \triangledown_{\theta} \log P(y^n|x^n;\theta)$) : $x^n$이 주어질 때 $y^n$의 확률을 최대로 하는 $\theta$ 찾는다.
  * 두 번째 항(${1\over K} \sum_{s=1}^S \sum_{i=1}^K \triangledown_\theta \log P(y^s|x_i;\theta)$) : 샘플링된 문장 $x^i$이 주어질 때, 단일 언어 코퍼스의 문장 $y^s$가 나올 평균 확률을 최대로 하는 $\theta$ 찾는다.  
  $\therefore$ back-translation 수행 $\leftrightarrow$ $\tilde{\mathcal{L}}(\theta)$ 최소화